In [16]:
#importing the Libraies
import numpy as np
import pandas as pd

In [15]:
# Reading the Dataset
dataset = pd.read_csv('/content/insurance_pre.csv')

In [17]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [18]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [19]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [20]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [21]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = {
    'n_estimators': [1, 5, 10],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],

}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3 ,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [1, 5, 10]},
             verbose=3)

In [30]:
# Assuming 'grid' is your GridSearchCV object and has been fitted to the data
print("Best parameters:", grid.best_params_)
# Access the mean test score for the best parameters
best_index = grid.best_index_
best_r2_score = grid.cv_results_['mean_test_score'][best_index]
# Printing the best R² score
print("r2:",best_r2_score)

Best parameters: {'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 10}
r2: 0.8307553611920984


In [31]:
table=pd.DataFrame.from_dict(grid.cv_results_)

In [32]:
table.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
59,0.174642,0.012749,0.004852,0.000209,absolute_error,10,2,5,10,"{'criterion': 'absolute_error', 'max_depth': 1...",0.863705,0.841021,0.823485,0.840621,0.784944,0.830755,0.026229,1
71,0.176555,0.004957,0.004918,0.000733,absolute_error,20,2,5,10,"{'criterion': 'absolute_error', 'max_depth': 2...",0.860027,0.830181,0.821753,0.850129,0.774675,0.827353,0.029667,2
47,0.175280,0.004066,0.004972,0.000169,absolute_error,None,2,5,10,"{'criterion': 'absolute_error', 'max_depth': N...",0.862862,0.841670,0.817982,0.837481,0.773801,0.826759,0.030074,3
56,0.173449,0.003376,0.005293,0.000904,absolute_error,10,2,2,10,"{'criterion': 'absolute_error', 'max_depth': 1...",0.858184,0.826363,0.811859,0.853175,0.769283,0.823773,0.032145,4
95,0.092810,0.004409,0.011235,0.003813,poisson,10,2,5,10,"{'criterion': 'poisson', 'max_depth': 10, 'min...",0.859604,0.825390,0.804740,0.855852,0.766754,0.822468,0.034411,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,0.040251,0.009516,0.004362,0.002095,absolute_error,None,1,2,1,"{'criterion': 'absolute_error', 'max_depth': N...",0.775528,0.631693,0.662779,0.667157,0.592602,0.665952,0.060947,104
72,0.009254,0.001198,0.002823,0.000117,poisson,None,1,2,1,"{'criterion': 'poisson', 'max_depth': None, 'm...",0.596808,0.671950,0.582551,0.746699,0.707733,0.661148,0.063124,105
60,0.022199,0.000676,0.002956,0.000061,absolute_error,20,1,2,1,"{'criterion': 'absolute_error', 'max_depth': 2...",0.671040,0.681510,0.590790,0.641501,0.681482,0.653265,0.034501,106
24,0.017070,0.005353,0.004374,0.003172,squared_error,20,1,2,1,"{'criterion': 'squared_error', 'max_depth': 20...",0.706038,0.595824,0.598961,0.672462,0.656455,0.645948,0.042765,107


In [33]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [35]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))

In [41]:
# Predict on test data
y_pred = loaded_model.predict(X_test)

# Evaluate performance
from sklearn.metrics import mean_squared_error, r2_score

print("MSE:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

MSE: 16869685.778710514
R2 Score: 0.8939879986613625
